In [3]:
import numpy as np
import pandas as pd
import os

## Import excel files into DataFrames

In [38]:
data_folder = 'SNA DATA'
country_note = pd.read_excel(data_folder + r'\country_note.xlsx')
country_register = pd.read_excel(data_folder + r'\country_register.xlsx')
footnote = pd.read_excel(data_folder + r'\footnote.xlsx')
series = pd.read_excel(data_folder + r'\series.xlsx')
sna93_data = pd.read_csv(data_folder + r'\sna93_data.txt')
sna93_group = pd.read_excel(data_folder + r'\sna93_group.xlsx')
sna93_item = pd.read_excel(data_folder + r'\sna93_item.xlsx')
sna93_sub_group = pd.read_excel(data_folder + r'\sna93_sub_group.xlsx')
sna93_sub_item = pd.read_excel(data_folder + r'\sna93_sub_item.xlsx')
sub_series = pd.read_excel(data_folder + r'\sub_series.xlsx')

## Merge DataFrames with sna93_data, the main data table

In [39]:
sna93_data = sna93_data.merge(country_register, how='left', on='country_code', validate='m:1')
sna93_data = sna93_data.merge(series, how='left', on=['country_code', 'series_number'], validate='m:m')
sna93_data = sna93_data.merge(sna93_group, how='left', on='group_code', validate='m:1')
sna93_data = sna93_data.merge(sna93_item, how='left', on=['group_code', 'item_code'], validate='m:1')
sna93_data = sna93_data.merge(sna93_sub_group, how='left', on=['group_code', 'sub_group_code'], validate='m:1')
sna93_data = sna93_data.merge(sna93_sub_item, how='left', on=['group_code', 'sub_item_code'], validate='m:1')
sna93_data = sna93_data.merge(sub_series, how='left', on=['country_code', 'series_number', 'sub_series_number'],
                              validate='m:1')
display(sna93_data.head())

,sna_value_id,country_code,group_code,item_code,sub_item_code,fiscal_year,series_number,sub_series_number,data_source,is_official,...,sna_system,period_type,currency_name,group_name,sub_group_code,sna93_item_code,item_name,sub_group_name,sub_item_name,base_year
0,2416223,4,101,7,0,2002,20,0,1,1,...,1968,1,new afghanis,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
1,5617028,4,101,7,0,2002,100,0,1,1,...,1993,1,Afghanis,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
2,2167078,4,101,7,0,2003,20,0,1,1,...,1968,1,new afghanis,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
3,5617029,4,101,7,0,2003,100,0,1,1,...,1993,1,Afghanis,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
4,5617030,4,101,7,0,2004,100,0,1,1,...,1993,1,Afghanis,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN


## Merge country codes

In [40]:
country_codes = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")
country_codes = country_codes[["alpha-2", "alpha-3", "country-code"]]
display(country_codes.head())

,alpha-2,alpha-3,country-code
0,AF,AFG,4
1,AX,ALA,248
2,AL,ALB,8
3,DZ,DZA,12
4,AS,ASM,16


In [41]:
sna93_data = sna93_data.merge(country_codes, how="left", left_on="country_code", right_on="country-code",
                validate="m:1")
# Observations missing alpha-2 codes
sna93_data[sna93_data["alpha-2"].isnull()]["country_name"].value_counts()
sna93_data.head()

,sna_value_id,country_code,group_code,item_code,sub_item_code,fiscal_year,series_number,sub_series_number,data_source,is_official,...,group_name,sub_group_code,sna93_item_code,item_name,sub_group_name,sub_item_name,base_year,alpha-2,alpha-3,country-code
0,2416223,4,101,7,0,2002,20,0,1,1,...,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0
1,5617028,4,101,7,0,2002,100,0,1,1,...,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0
2,2167078,4,101,7,0,2003,20,0,1,1,...,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0
3,5617029,4,101,7,0,2003,100,0,1,1,...,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0
4,5617030,4,101,7,0,2004,100,0,1,1,...,Table 1.1 Gross domestic product by expenditur...,1,P.3,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0


## Convert Currencies

In [45]:
exchange_rates = pd.read_csv(data_folder + r"\exchange_rates_bis.csv")
exchange_rates = exchange_rates[(exchange_rates["FREQ"]=="A") & (exchange_rates["COLLECTION"]=="E")]
exchange_rates = exchange_rates.melt(id_vars="REF_AREA", value_vars=[str(n) for n in range(1940, 2019)], value_name="exchange_rate")
exchange_rates = exchange_rates.rename(columns={"variable": "Year"})
exchange_rates["Year"] = exchange_rates["Year"].astype(int)
display(exchange_rates[~exchange_rates["exchange_rate"].isnull()].head())

,REF_AREA,Year,exchange_rate
8,AR,1940,4.000000e-13
24,BR,1940,6.048000e-15
25,BS,1940,1.020410e+00
57,FM,1940,1.000000e+00
134,PA,1940,1.000000e+00


In [46]:
sna93_data = sna93_data.merge(exchange_rates, how="left", left_on=["alpha-2", "fiscal_year"],
                             right_on=["REF_AREA", "Year"], validate="m:m")
sna93_data["sna_value_dollars"] = sna93_data["sna_value"] / sna93_data["exchange_rate"]
sna93_data.head()

,sna_value_id,country_code,group_code,item_code,sub_item_code,fiscal_year,series_number,sub_series_number,data_source,is_official,...,sub_group_name,sub_item_name,base_year,alpha-2,alpha-3,country-code,REF_AREA,Year,exchange_rate,sna_value_dollars
0,2416223,4,101,7,0,2002,20,0,1,1,...,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0,AF,2002.0,47.263,4.232529e+09
1,5617028,4,101,7,0,2002,100,0,1,1,...,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0,AF,2002.0,47.263,5.092313e+09
2,2167078,4,101,7,0,2003,20,0,1,1,...,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0,AF,2003.0,48.865,4.653065e+09
3,5617029,4,101,7,0,2003,100,0,1,1,...,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0,AF,2003.0,48.865,6.682595e+09
4,5617030,4,101,7,0,2004,100,0,1,1,...,Expenditures of the gross domestic product,NaN,NaN,AF,AFG,4.0,AF,2004.0,48.220,7.234363e+09


In [48]:
sna93_data.columns

Index(['sna_value_id', 'country_code', 'group_code', 'item_code',
       'sub_item_code', 'fiscal_year', 'series_number', 'sub_series_number',
       'data_source', 'is_official', 'sna_value', 'country_name', 'sna_system',
       'period_type', 'currency_name', 'group_name', 'sub_group_code',
       'sna93_item_code', 'item_name', 'sub_group_name', 'sub_item_name',
       'base_year', 'alpha-2', 'alpha-3', 'country-code', 'REF_AREA', 'Year',
       'exchange_rate', 'sna_value_dollars'],
      dtype='object')

In [58]:
sna93_data_small = sna93_data[['country_code', 'fiscal_year', 'data_source', 'sna_value_dollars',
                              'country_name', 'sna_system', 'period_type', 'currency_name',
                              'group_name', 'item_name', 'sub_group_name', 'sub_item_name',
                              'base_year']]
print(sna93_data_small.shape)
sna93_data_small.head()

(3612794, 13)


,country_code,fiscal_year,data_source,sna_value_dollars,country_name,sna_system,period_type,currency_name,group_name,item_name,sub_group_name,sub_item_name,base_year
0,4,2002,1,4.232529e+09,Afghanistan,1968,1,new afghanis,Table 1.1 Gross domestic product by expenditur...,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
1,4,2002,1,5.092313e+09,Afghanistan,1993,1,Afghanis,Table 1.1 Gross domestic product by expenditur...,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
2,4,2003,1,4.653065e+09,Afghanistan,1968,1,new afghanis,Table 1.1 Gross domestic product by expenditur...,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
3,4,2003,1,6.682595e+09,Afghanistan,1993,1,Afghanis,Table 1.1 Gross domestic product by expenditur...,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN
4,4,2004,1,7.234363e+09,Afghanistan,1993,1,Afghanis,Table 1.1 Gross domestic product by expenditur...,Final consumption expenditure,Expenditures of the gross domestic product,NaN,NaN


## Finanacial Account, Portfolio Investment, Assets/Liabilities, Equity Series

In [59]:
sna93_data_key = sna93_data_small[sna93_data_small["sub_group_name"].str.contains("Financial account", na=False, regex=False)]
key_country_codes = ["840", "124", "484", "76", "152", "32", "604", "826", "250", "276", "724", "380",
                     "756", "442", "156", "392", "344", "702", "410", "36", "554"]
sna93_data_key = sna93_data_key[sna93_data_key["country_code"].isin(key_country_codes)]
display(sna93_data_key["country_name"].value_counts()) # Missing Argentina and Peru
display(sna93_data_key.head())

Australia                                         7558
Spain                                             7246
Canada                                            6679
France                                            6619
Germany                                           6336
Italy                                             5788
United States                                     5630
United Kingdom                                    5557
Republic of Korea                                 4668
Mexico                                            4227
Luxembourg                                        3368
Chile                                             2767
Japan                                             2681
Switzerland                                       1379
Singapore                                          975
China                                              618
Brazil                                             605
New Zealand                                        124
China, Hon

,country_code,fiscal_year,data_source,sna_value_dollars,country_name,sna_system,period_type,currency_name,group_name,item_name,sub_group_name,sub_item_name,base_year
124759,36,1988,2,2.808573e+10,Australia,1993,3,Australian dollar,Table 4.1 Total Economy (S.1),Net incurrence of liabilities,III.2 Financial account - Changes in liabiliti...,NaN,NaN
124760,36,1988,2,2.808573e+10,Australia,1993,3,Australian dollar,Table 4.1 Total Economy (S.1),Net incurrence of liabilities,III.2 Financial account - Changes in liabiliti...,NaN,NaN
124761,36,1989,2,2.537160e+10,Australia,1993,3,Australian dollar,Table 4.1 Total Economy (S.1),Net incurrence of liabilities,III.2 Financial account - Changes in liabiliti...,NaN,NaN
124762,36,1989,2,2.537160e+10,Australia,1993,3,Australian dollar,Table 4.1 Total Economy (S.1),Net incurrence of liabilities,III.2 Financial account - Changes in liabiliti...,NaN,NaN
124763,36,1989,2,1.067541e+11,Australia,1993,3,Australian dollar,Table 4.1 Total Economy (S.1),Net incurrence of liabilities,III.2 Financial account - Changes in liabiliti...,NaN,NaN


In [60]:
pivot_columns = ['sna_system', 'group_name', 'sub_group_name', 'item_name', 'data_source',
                'period_type']
sna93_data_key_wide = sna93_data_key.pivot_table(index=['country_name', 'fiscal_year'],
                                                      columns=pivot_columns,
                                                      values='sna_value_dollars',
                                                      aggfunc='sum')
sna93_data_key_wide

sna_system                                                         1968  \
group_name                            Table 4.2 Rest of the world (S.2)   
sub_group_name            V.III.2 Financial account - Changes in assets   
item_name                           Net acquisition of financial assets   
data_source                                                           0   
period_type                                                           0   
country_name  fiscal_year                                                 
Australia     1988                                                  NaN   
              1989                                                  NaN   
              1990                                                  NaN   
              1991                                                  NaN   
              1992                                                  NaN   
...                                                                 ...   
United States 2013                                                  NaN   
              2014                                                  NaN   
              2015                                                  NaN   
              2016                                                  NaN   
              2017                                                  NaN   

sna_system                                                                                  \
group_name                                                                                   
sub_group_name            V.III.2 Financial account - Changes in liabilities and net worth   
item_name                                              NET LENDING (+) / NET BORROWING (-)   
data_source                                                                              0   
period_type                                                                              0   
country_name  fiscal_year                                                                    
Australia     1988                                                       NaN                 
              1989                                                       NaN                 
              1990                                                       NaN                 
              1991                                                       NaN                 
              1992                                                       NaN                 
...                                                                                    ...   
United States 2013                                                       NaN                 
              2014                                                       NaN                 
              2015                                                       NaN                 
              2016                                                       NaN                 
              2017                                                       NaN                 

sna_system                                               \
group_name                                                
sub_group_name                                            
item_name                 Net incurrence of liabilities   
data_source                                           0   
period_type                                           0   
country_name  fiscal_year                                 
Australia     1988                                  NaN   
              1989                                  NaN   
              1990                                  NaN   
              1991                                  NaN   
              1992                                  NaN   
...                                                 ...   
United States 2013                                  NaN   
              2014                                  NaN   
              2015                                  NaN   
            

In [71]:
# Export to csv
# sna93_data_key_wide.to_csv("key_series.csv")

## Convert data from long to wide

In [61]:
sna93_data_table4 = sna93_data_small[sna93_data_small["group_name"].str.startswith("Table 4")]
sna93_data_table4 = sna93_data_table4.dropna(axis=1, how='all')
sna93_data_table4.head()

,country_code,fiscal_year,data_source,sna_value_dollars,country_name,sna_system,period_type,currency_name,group_name,item_name,sub_group_name
17534,20,1996,1,NaN,Andorra,1968,0,Euro,Table 4.2 Rest of the world (S.2),Imports of goods and services,V.I External account of goods and services - R...
17535,20,1997,1,NaN,Andorra,1968,0,Euro,Table 4.2 Rest of the world (S.2),Imports of goods and services,V.I External account of goods and services - R...
17536,20,1998,1,NaN,Andorra,1968,0,Euro,Table 4.2 Rest of the world (S.2),Imports of goods and services,V.I External account of goods and services - R...
17537,20,1999,1,NaN,Andorra,1968,0,Euro,Table 4.2 Rest of the world (S.2),Imports of goods and services,V.I External account of goods and services - R...
17538,20,2000,1,NaN,Andorra,1968,0,Euro,Table 4.2 Rest of the world (S.2),Imports of goods and services,V.I External account of goods and services - R...


In [62]:
pivot_columns = ['group_name', 'sub_group_name', 'item_name', 'currency_name', 'sna_system', 'data_source',
                'period_type']
sna93_data_table4_wide = sna93_data_table4.pivot_table(index=['country_name', 'fiscal_year'],
                                                      columns=pivot_columns,
                                                      values='sna_value_dollars',
                                                      aggfunc='sum')
sna93_data_table4_wide

group_name                                                                                                       Table 4.1 Total Economy (S.1)  \
sub_group_name                                                                                               I. Production account - Resources   
item_name                Less: Financial intermediation services indirectly measured (only to be deducted if FISIM is not distributed to uses)   
currency_name                                                                                                                  (second) Kwanza   
sna_system                                                                                                                                1993   
data_source                                                                                                                                  1   
period_type                                                                                                                                  0   
country_name fiscal_year                                                                                                                         
Andorra      1996                                                       NaN                                                                      
             1997                                                       NaN                                                                      
             1998                                                       NaN                                                                      
             1999                                                       NaN                                                                      
             2000                                                       NaN                                                                      
...                                                                                                                                        ...   
Yemen        2012                                                       NaN                                                                      
             2013                                                       NaN                                                                      
             2014                                                       NaN                                                                      
             2015                                                       NaN                                                                      
             2016                                                       NaN                                                                      

group_name                                                          \
sub_group_name                                                       
item_name                                                            
currency_name            1999 ATS euro / euro 1999 BEF euro / euro   
sna_system                               1993                 1993   
data_source                                 2                    2   
period_type                                 0                    0   
country_name fiscal_year                                             
Andorra      1996                         NaN                  NaN   
             1997                         NaN                  NaN   
             1998                         NaN                  NaN   
             1999                         NaN                  NaN   
             2000                         NaN                  NaN   
...                                       ...                  ...   
Yemen        2012                         NaN                  NaN   
             2013                         NaN                  NaN   
             2014                         NaN                  NaN   
             2015                  